#### Generate ideas based on a similarity search stored in a vectordb

- idea is we grab the vectordb stored locally. if none, exists just create an empty one.
- provide that list of ideas i guess? maybe to the ollama prompt and tell it like hey(in a rlly good prompt) that you want to generate a script for instagram reel on a new topic that i ahve not gone over yet from this vectordb store.
- prvoide script

In [1]:
from langchain_community.llms import Ollama # will be used for prompting
from langchain.vectorstores import Chroma # will be used for vectordb store
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document # will be used to store text in vector store 
import os

Use the same model for embedding text (to compare idea similarity). Instantiate the local Ollama model (LLaMA3 in this case)

In [2]:
llm = Ollama(model="llama3")
embedding = OllamaEmbeddings(model="llama3")

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_3404/2493284979.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [3]:
PERSIST_DIR = "vectordb"

Loading vector DB
- supposed to be a function that loads the vector_db
- if it dne, create one
- create an instance of a vector db using chroma with that directory created(or used prev) and the embedding attached too

In [4]:
if not os.path.exists(PERSIST_DIR):
    os.makedirs(PERSIST_DIR)

db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_3404/1878142402.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embedding)


This function will return a list of all prev saved ideas from the DB
- it'll run a similarity search on the db with the key "" --> itll return all documents (ideas)
- we will just limit to a 100 for now 
- also, this should be its own function that returns an array, but for testing purposes, ill just append to an array

In [5]:
past_ideas = []
docs = db.similarity_search("", k=100)
for doc in docs:
    past_ideas.append(doc.page_content)

Saving a new idea
- i am going to create a function for this now thatll be used later in the query function i am about to run 
- simple idea. create a document object with the idea as the page_content
- then add the document to the db object init earlier
- dont forget the persist!


UPDATE - adding meta data such as a unique id, timestamp (going to be used for getting the most recent), processed (so we can query and not have a large output), and the topic 

In [ ]:
def save_new_idea(db, idea_text):
    # Create metadata
    unique_id = str(uuid.uuid4())
    timestamp = datetime.utcnow().isoformat()

    metadata = {
        "id": unique_id,
        "timestamp": timestamp,
        "processed": False
    }

    # Create Document object with metadata
    doc = Document(page_content=idea_text, metadata=metadata)

    # Add to DB and persist
    db.add_documents([doc])
    db.persist()

    print(f"Idea saved with ID: {unique_id}")

Generate new idea function:
- this function will take in past_ideas as a parameter. and it join them as a string with a new line character per idea. if no ideas in the past_ideas array, itll set joined_ideas as None yet.
- the prompt will be well strucutred as an f string thatll hold the joined_ideas var in it
- the llm will be invoked with teh prompt and expect a response

In [7]:
def generate_new_idea(past_ideas):
    # Combine all past ideas into a single string, or show "None yet" if empty
    joined_ideas = "\n".join(past_ideas) if past_ideas else "None yet."

    # Craft a well-structured prompt asking for a creative, new idea
    prompt = f"""
You are a creative AND technical Instagram assistant. Below is a list of past Instagram video topics:

{joined_ideas}

Your task is to come up with a completely new and original **60-second Instagram script** idea related to AWS services that hasn’t been covered yet.

Make the script informative, engaging, and structured as follows:
1. Hook: Catchy opening line to grab attention.
2. What it is: Explain the AWS service in plain English.
3. When to use it: Mention typical use cases or situations.
4. How to use it (high-level): Describe in 1 simple line how a developer might use or set it up.
5. Bonus tip or cool fact (optional): Share something surprising or helpful.
6. Call to Action: Prompt viewers to engage.

Use a casual tone like you're talking to a tech-savvy friend. Avoid repeating old topics from the list above.
"""


    # Send prompt to the LLaMA model and get a generated response
    return llm.invoke(prompt)

Since we already kinda init the db (and loaded old prev data too if there even existed in the first place), im not going to include it in the function below. however, ill still include the generate_new_idea function since this is actually calling the llm and will also call the save_new_idea function

In [8]:
def run_idea_generation(db):
    new_idea = generate_new_idea(past_ideas)  # Generate a new idea
    save_new_idea(db, new_idea)           # Save the new idea to the DB
    return new_idea

In [9]:
print(run_idea_generation(db))

Here's a new Instagram script idea:

**Title:** "Level Up Your Machine Learning Models with AWS SageMaker Autopilot"

**Hook:** "Are you tired of manually tuning machine learning models? Well, we've got some game-changing news for you!"

**What it is:** "AWS SageMaker Autopilot is an automated service that helps you optimize your machine learning models without needing to write a single line of code. It's like having a personal data scientist assistant!"

**When to use it:** "Autopilot is perfect for situations where you're working with large datasets and need to quickly train and deploy accurate models. Think predictive maintenance, customer churn prediction, or natural language processing."

**How to use it (high-level):** "Just upload your dataset, select a pre-built model, and let Autopilot do the rest! You can even get real-time feedback on performance metrics to continuously improve your models."

**Bonus tip or cool fact:** "Did you know that SageMaker Autopilot uses reinforceme

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_3404/2143871773.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist() # persist changes to disk
